# Short-term residential load forecasting with Deep Learning

London Households SmartMeter Data

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
import IPython
import IPython.display
import glob
import time
import warnings
warnings.filterwarnings("ignore")
randomState = 42 # tip of the cap to Douglas Adams
#!pip install -U pip
#!pip install -U setuptools wheel

#!pip install autogluon


Creat two load forecasts...
1) half-hourly load forecast for next 24 hours
2) peak half-hour in the next 24 hours

Value...
* For electric network operator, minimize the amount of spinning reserve

Data...
* residential smart meter usage data
* weather data
* weather forecast data

# Load Data

## Load half-hourly electric usage data
...for ~5k smart meters in London
[SmartMeter Energy Consumption Data in London Households](https://data.london.gov.uk/dataset/smartmeter-energy-use-data-in-london-households)

In [ ]:
import time
# load half-hourly electric usage data
# takes about four minutes, need to find somerthing faster like Dask?
# https://data.london.gov.uk/dataset/smartmeter-energy-use-data-in-london-households
d = pd.read_csv('/kaggle/input/small-lcl-data/LCL-June2015v2_99.csv', parse_dates=["DateTime"])

# Get CSV files list from a folder
path = '/kaggle/input/smart-meters-in-london/halfhourly_dataset/halfhourly_dataset'
csv_files = glob.glob(path + "/*.csv")

# Read each CSV file into DataFrame
# This creates a list of dataframes
start_time = time.time()
df_list = (pd.read_csv(file, parse_dates=["tstp"]) for file in csv_files)
print('%s seconds' % (time.time() - start_time))

# Concatenate all DataFrames
start_time = time.time()
d = pd.concat(df_list, ignore_index=True)
print('%s seconds' % (time.time() - start_time))

In [ ]:
d.describe()

## Load hourly weather data

In [ ]:
# load hourly weather data
# https://data.london.gov.uk/dataset/smartmeter-energy-use-data-in-london-households
weatherData = pd.read_csv('/kaggle/input/smart-meters-in-london/weather_hourly_darksky.csv', parse_dates=["time"])

In [ ]:
weatherData.describe()

In [ ]:
weatherData.info()

# Data pre-processing and cleaning

## Weather data: convert text attributes to string datatype

In [ ]:
weatherData = weatherData.astype({'precipType':'string', 'icon':'string', 'summary':'string'})


In [ ]:
from pandas_profiling import ProfileReport

profile = ProfileReport(weatherData, tsmode=True, sortby="time")
profile.to_file('weatherData profile_report.html')
# profile


## Identify and remove weather records not exactly on the hour

In [ ]:
# inspect and remove records not exactly on the hour
offRecs = weatherData.query("time.dt.minute != 0 or time.dt.second != 0")
print('Records not exactly on the half-hour:\n ', offRecs)

## Upsample weather data to match half-houly sampling rate of load data

In [ ]:
# select weather data features of interest
weatherUpsample = weatherData[['time','temperature', 'dewPoint', 'pressure', 'humidity']].copy()
weatherUpsample = weatherUpsample.sort_values(by=['time'])
print(weatherUpsample.info())
print(weatherUpsample.describe())
print(weatherUpsample)

weatherUpsample = weatherUpsample.set_index('time')
weatherUpsample.index.rename('time', inplace=True)

start_time = time.time()

weatherUpsample = weatherUpsample.resample('30Min').mean()

# upsample 
weatherUpsample['temperature'] = weatherUpsample['temperature'].interpolate()
weatherUpsample['dewPoint'] = weatherUpsample['dewPoint'].interpolate()
weatherUpsample['pressure'] = weatherUpsample['pressure'].interpolate()
weatherUpsample['humidity'] = weatherUpsample['humidity'].interpolate()

print('%s seconds' % (time.time() - start_time))

weatherUpsample = weatherUpsample.reset_index(names='DateTime')
print(weatherUpsample.info())
print(weatherUpsample.describe())
print(weatherUpsample)

In [ ]:
weatherUpsample.to_csv('/kaggle/working/WeatherDataFinal.csv',index=False)

In [ ]:
weatherUpsample = pd.read_csv('/kaggle/working/WeatherDataFinal.csv', parse_dates=["DateTime"])
weatherUpsample

In [ ]:
d.info()

In [ ]:
# utility function to nicely format variable names and memory they are consuming
import sys
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

## Convert smart meter usage datatype to float

In [ ]:
# ~1 minute
start_time = time.time()
d.iloc[:, 2] = pd.to_numeric(d.iloc[:, 2], errors='coerce')
print('%s seconds' % (time.time() - start_time))

# rename usage column for easier reference
d.rename(columns={d.columns[2]: 'KWHperHH'}, inplace=True)
d.info()

In [ ]:
# set timestamp as the index
start_time = time.time()
d.set_index('tstp')
print('%s seconds' % (time.time() - start_time))

## Identify and handle duplicates in the smart meter data

In [ ]:
# about 1.5 minutes
start_time = time.time()
dupes = d[d.duplicated()]
print('dupes', dupes)
print('dupes.index', dupes.index)
d.drop(index=dupes.index, inplace=True)
print('%s seconds' % (time.time() - start_time))

In [ ]:
# set index for the usage data to the timestamp column.  Is this necessary?  Can't remember why
start_time = time.time()
d.set_index('tstp')
d.info()
print('%s seconds' % (time.time() - start_time))

In [ ]:
# check what is gobbling RAM
for name, size in sorted(((name, sys.getsizeof(value)) for name, value in list(
                          locals().items())), key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))


# Visualize smart meter dataset to analyze for quality, completenes and othe insights

In [ ]:
import seaborn as sns

## grab a random sample of 2% of meters for visualization and analysis

In [ ]:
rng = np.random.default_rng(randomState)
# random_state = np.random.RandomState(randomState)
sampleMeters = rng.choice(d.LCLid.unique(), size=int(len(d.LCLid.unique())*0.02), replace=False)
# sampleMeters = np.random.choice(d.LCLid.unique(), size=int(len(d.LCLid.unique())*0.02), replace=False, random_state=random_state)
print('sampleMeters:\n', sampleMeters)
sample = d[d['LCLid'].isin(sampleMeters)]
print('sample:\n', sample)
print(sample.describe())
# print(sample.info())

## Heatmap to visualize meter read coverage and completeness

In [ ]:
# visualize meter read coverage and completeness
# using a random sample of 2% of meters
plt.subplots(figsize=(20,15))
pivot_table = pd.pivot_table(sample, columns='tstp', index='LCLid', values='KWHperHH')
sns.heatmap(pivot_table)
plt.savefig('meter data heatmap.png', format='png')


Observations from Heatmap...
* several houses start producing load part-way through the period
    - eg MAC004221, MAC004248
    
    
* several houses stop producing part-way through the period
    - eg MAC004226, MAC004257
    

* most houses have at least one "gap" in their data (visible as white lines)


* several houses stand out as having significantly higher average load than others
    - eg MAC004225, MAC004249

## identify and remove smart meter readings not exactly on the half-hour

In [ ]:
# identify and remove records not exactly on the half-hour

start_time = time.time()

offRecs = d.query("tstp.dt.minute not in (0,30) or tstp.dt.second != 0")
# aggLoad["DateTime"].dt.hour > 30
print('Records not exactly on the half-hour:\n ', offRecs)
print(offRecs.info())

# delete records not exactly on the half-hour
d.drop(offRecs.index, inplace=True)

print('%s seconds' % (time.time() - start_time))

offRecs = d.query("tstp.dt.minute not in (0,30) or tstp.dt.second != 0")
print('Records not exactly on the half-hour: ', offRecs)

In [ ]:
d.info()

In [ ]:
# check what is gobbling RAM
for name, size in sorted(((name, sys.getsizeof(value)) for name, value in list(
                          locals().items())), key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))


## Fill gaps in the smart meter data using interpolation

Our heatmap above shows lots of gaps (small white vertical lines), and we'll fill those gaps using interpolation

### First step of filling these gaps is to create NaN records where records are missing

Then we can fill these gas with interpolation

In [ ]:
# First step of interpolation is to create NaN records where records are missing
# about 2 minutes
d.sort_values(by=['tstp'], inplace=True)
d.set_index('tstp', inplace=True)
d.index.rename('tstp', inplace=True)

start_time = time.time()
# resample to create NaN records where records are missing
d = d.groupby('LCLid')\
                .resample('30Min')\
                .mean()

# fill the gaps with interpolation
d['KWHperHH'] = d['KWHperHH'].interpolate()
d.reset_index(inplace=True)

print('%s seconds' % (time.time() - start_time))

## Check the meter data heatmap to see if gaps have been filled

In [ ]:
# visualize after interpolating missing values
d.info()
sample = d[d['LCLid'].isin(sampleMeters)]
pivot_table = pd.pivot_table(sample, columns='tstp', index='LCLid', values='KWHperHH')
plt.subplots(figsize=(20,15))

sns.heatmap(pivot_table)
plt.savefig('meter data heatmap gaps filled.png', format='png')

## Visualize zeros in the dataset using heatmap

I'm always curious to understand zeros in a dataset, and whether they are legitimate zero values, or indicate a data quality problem.

In [ ]:
# visualize zeros in the dataset
start_time = time.time()
sample = d[d['LCLid'].isin(sampleMeters)]
sample['ZeroKWHperHH'] = sample['KWHperHH'] == 0
pivot_table = pd.pivot_table(sample, columns='tstp', index='LCLid', values='ZeroKWHperHH')
print('%s seconds' % (time.time() - start_time))
plt.subplots(figsize=(20,15))

sns.heatmap(pivot_table)
plt.savefig('meter data heatmap zeros.png', format='png')

In [ ]:
# take a snapshot of data
start_time = time.time()
d.to_csv('/kaggle/working/MeterDataFinal.csv',index=False)
print('%s seconds' % (time.time() - start_time))

### Obervation

There are a handful of households that account all the zero value meter reads: MAC004233, MAC004226, MAC004267

In [ ]:
# investigate the meters with zero reads
MAC005127 = sample.query("LCLid == 'MAC005127'")

fig, ax = plt.subplots(4,figsize=(20,9))

# plot whole ~2 years
ax[0].plot(MAC005127.tstp, MAC005127.KWHperHH)
ax[0].plot(MAC005127.tstp, MAC005127.ZeroKWHperHH)
ax[0].set(ylabel='KWH/hh',
       title='Load from one Household MAC004233 with lots of zero values')
plt.tick_params(rotation=45)
ax[0].grid()

# zoom in
ax[1].plot(MAC005127.tstp[11000:15000], MAC005127.KWHperHH[11000:15000])
ax[1].plot(MAC005127.tstp[11000:15000], MAC005127.ZeroKWHperHH[11000:15000])
ax[1].set(xlabel='time (s)', ylabel='KWH/hh')
plt.tick_params(rotation=45)
ax[1].grid()

# zoom in more...
ax[2].plot(MAC005127.tstp[13000:13500], MAC005127.KWHperHH[13000:13500])
ax[2].plot(MAC005127.tstp[13000:13500], MAC005127.ZeroKWHperHH[13000:13500])
ax[2].set(xlabel='time (s)', ylabel='KWH/hh')
plt.tick_params(rotation=45)
ax[2].grid()

# zoom in to a different part of the series...
ax[3].plot(MAC005127.tstp[25000:25500], MAC005127.KWHperHH[25000:25500])
ax[3].plot(MAC005127.tstp[25000:25500], MAC005127.ZeroKWHperHH[25000:25500])
ax[3].set(xlabel='time (s)', ylabel='KWH/hh')
plt.tick_params(rotation=45)
ax[3].grid()

fig.savefig("MAC005127.png")
plt.show()

### Observation:
The zeros for MAC004233 seem legit - leaving them in

In [ ]:
# investigate the meters with zero reads
MAC002304 = sample.query("LCLid == 'MAC002304'")
fig, ax = plt.subplots(4,figsize=(20,9))

# plot whole ~2 years
ax[0].plot(MAC002304.tstp, MAC002304.KWHperHH)
ax[0].plot(MAC002304.tstp, MAC002304.ZeroKWHperHH)
ax[0].set(ylabel='KWH/hh',
       title='Load from one Household MAC002304 with lots of zero values')
plt.tick_params(rotation=45)
ax[0].grid()

# zoom in
ax[1].plot(MAC002304.tstp[17000:21000], MAC002304.KWHperHH[17000:21000])
ax[1].plot(MAC002304.tstp[17000:21000], MAC002304.ZeroKWHperHH[17000:21000])
ax[1].set(xlabel='time (s)', ylabel='KWH/hh')
plt.tick_params(rotation=45)
ax[1].grid()

# zoom in more...
ax[2].plot(MAC002304.tstp[19300:19800], MAC002304.KWHperHH[19300:19800])
ax[2].plot(MAC002304.tstp[19300:19800], MAC002304.ZeroKWHperHH[19300:19800])
ax[2].set(xlabel='time (s)', ylabel='KWH/hh')
plt.tick_params(rotation=45)
ax[2].grid()

# zoom in to a different part of the series...
ax[3].plot(MAC002304.tstp[25000:25500], MAC002304.KWHperHH[25000:25500])
ax[3].plot(MAC002304.tstp[25000:25500], MAC002304.ZeroKWHperHH[25000:25500])
ax[3].set(xlabel='time (s)', ylabel='KWH/hh')
plt.tick_params(rotation=45)
ax[3].grid()

fig.savefig("MAC002304.png")
plt.show()

### Observation

The zeros for MAC004233 seem legit - leaving them in


In [ ]:
# visualize and handle outliers

# minumum and maximum timestamp for each house
print(d.groupby('LCLid').max().sort_values('tstp'))
print(d.groupby('LCLid').min().sort_values('tstp'))
print(d.groupby('LCLid').count().sort_values('tstp'))

print(d.groupby('LCLid').agg(['min', 'max', 'count']))


In [ ]:
# which house has the highest peak load?

# which house has the highest total aggregate load?

# how variable / predictable is the timing of the peak load

# how accurate is the next 24 hours forecast profile overall?

# how accurate is the peak load forecast in next 24 hours?

# normalize and standardize


In [ ]:
# extract one smartmeter for plotting
sample = d.query("LCLid == 'MAC004233'")
sample

In [ ]:
# Visualize load profile for one household meter
fig, ax = plt.subplots()
ax.plot(sample.iloc[100:4500,1], sample.iloc[100:4500,2])

ax.set(xlabel='time (s)', ylabel='KWH/hh',
       title='Load from one Household, June-September 2012')
plt.tick_params(rotation=45)
ax.grid()

fig.savefig("Load from one Household, June-September 2012.png")
plt.show()

In [ ]:
# set index for the sample
sample.set_index('tstp')

# EDA: Visualize daily average load for each meter and all meters...

In [ ]:
# calculate average daily load profile for all meters...
# abut 1.5 minutes

start_time = time.time()
avgLoadProfile = pd.DataFrame(d.groupby([d['tstp'].dt.hour, d['tstp'].dt.minute]).KWHperHH.mean())
avgLoadProfile = avgLoadProfile.reset_index(names=['hour', 'minute'])
avgLoadProfile['labels'] = pd.to_datetime(avgLoadProfile['hour'].astype(str) + ':' + avgLoadProfile['minute'].astype(str), format='%H:%M').dt.time
print('%s seconds' % (time.time() - start_time))

# print(avgLoadProfile.info())
# print(avgLoadProfile)

fig, ax = plt.subplots(figsize=(10,7))

ax.set_xticks(avgLoadProfile.index, avgLoadProfile.labels)

ax.set(xlabel='time (HH:MI)', ylabel='KWH/hh',
       title='Average Household 24 hour load profile')

# calculate average daily load for each meter...
start_time = time.time()
avgLoadProfileEachMeter = pd.DataFrame(d.groupby(['LCLid', d['tstp'].dt.hour, d['tstp'].dt.minute]).agg({'KWHperHH': 'mean'}))
avgLoadProfileEachMeter = avgLoadProfileEachMeter.reset_index(names=['LCLid', 'hour', 'minute'])
print('%s seconds' % (time.time() - start_time))
# print(avgLoadProfileEachMeter.info())
# print(avgLoadProfileEachMeter)

# plot every sample meter
for meter in sampleMeters:
    # print(meter)
    ax.plot(avgLoadProfileEachMeter.loc[avgLoadProfileEachMeter['LCLid'] == meter].index % 48, 
            avgLoadProfileEachMeter.loc[avgLoadProfileEachMeter['LCLid'] == meter].KWHperHH,
           color='grey')

# plot the average
ax.plot(avgLoadProfile.index, avgLoadProfile.KWHperHH, linewidth=5)

plt.tick_params(rotation=45)
ax.grid()

fig.savefig("Avg 24hr Load Profile every meter.png")
plt.show()

In [ ]:
# Calculate the sum of all loads for each timestamp using `groupby()` and `agg()`
start_time = time.time()
# aggLoad = d.groupby('tstp')['KWHperHH'].agg('sum')
aggLoad = pd.DataFrame(d.groupby('tstp')['KWHperHH'].agg({'sum', 'count'}))
aggLoad.reset_index(inplace=True)
aggLoad.columns = ['tstp', 'AggregateLoad', 'numMeters']
print('%s seconds' % (time.time() - start_time))

print(aggLoad)
print(aggLoad.describe())
print(aggLoad.info())

In [ ]:
aggLoad.sort_values(by=['tstp'], inplace=True)
aggLoad.set_index('tstp', inplace=True)
aggLoad.index.rename('DateTimeIndex', inplace=True)
aggLoad.info()

In [ ]:
aggLoad['DateTime'] = aggLoad.index
aggLoad.info()

In [ ]:
# inspect and fix records with zero load
# start with the aggregated records with zero load
AggZeros = aggLoad.query("AggregateLoad == 0")
AggZeros



Observation: Some of the timestamps are not exactly on the half-hour
Question: How many of the timestamps are not exactly on the half-hour?

In [ ]:
# inspect and fix records not exactly on the half-hour
offRecs = aggLoad.query("DateTime.dt.minute not in (0,30) or DateTime.dt.second != 0")
# aggLoad["DateTime"].dt.hour > 30
print('Records not exactly on the half-hour: ', offRecs)
print(offRecs.info())

# delete records not exactly on the half-hour
aggLoad = aggLoad.drop(offRecs.index)

offRecs = aggLoad.query("DateTime.dt.minute not in (0,30) or DateTime.dt.second != 0")
print('Records not exactly on the half-hour: ', offRecs)

In [ ]:
# check the regularity of the observations (time between observations)
# print(pd.infer_freq(train_data.DateTime))
aggLoad.index.to_series().diff().value_counts()

In [ ]:
# Calculate moving average and stddev
window_size = int(len(aggLoad.AggregateLoad) / 10)
print(window_size)

aggLoadMovingStdev = aggLoad.AggregateLoad.rolling(window_size).std()
aggLoadMovingStdev.columns = ['MovingStdev']
# aggLoadMovingStdev.columns.values[0] = 'MovingStdev'

aggLoadMovingAvg = aggLoad.AggregateLoad.rolling(window_size).mean()
aggLoadMovingAvg.columns = ['MovingAvg']

print('aggLoadMovingStdev:\n', aggLoadMovingStdev)
print(aggLoadMovingStdev.info())
print('aggLoadMovingAvg:\n', aggLoadMovingAvg)
print(aggLoadMovingAvg.info())

# aggLoad['MovingStdev'] = aggLoad.AggregateLoad.rolling(window_size).std()
# aggLoad['MovingAvg'] = aggLoad.AggregateLoad.rolling(window_size).mean()

# print('aggLoad.MovingStdev:\n', aggLoad.MovingStdev)
# print('aggLoad.MovingAvg:\n', aggLoad.MovingAvg)



In [ ]:
print(aggLoad)

fig, ax = plt.subplots(figsize=(20,7))
ax.plot(aggLoad.DateTime, aggLoad.AggregateLoad)
# ax.plot(aggLoad.DateTime, aggLoad.MovingAvg, linewidth=3)
# ax.plot(aggLoad.DateTime, aggLoad.MovingStdev, linewidth=3)
ax.plot(aggLoad.DateTime, aggLoadMovingAvg, linewidth=3)
ax.plot(aggLoad.DateTime, aggLoadMovingStdev, linewidth=3)

ax.set(xlabel='time (s)', ylabel='KWH/hh',
       title='Aggregate Household load 2012-2014')
plt.tick_params(rotation=45)
ax.grid()

fig.savefig("Aggregate Household load 2012-2014.png")
plt.show()

In [ ]:
aggLoad.head()


In [ ]:
fig, ax = plt.subplots(figsize=(20,7))
ax.plot(aggLoad.DateTime[10000:15000], aggLoad.AggregateLoad[10000:15000])

ax.set(xlabel='time (s)', ylabel='KWH/hh',
       title='Aggregate Household load June-August 2012')
plt.tick_params(rotation=45)
ax.grid()

fig.savefig("Aggregate Household load June-August 2012.png")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20,7))
ax.plot(aggLoad.DateTime[12000:13000], aggLoad.AggregateLoad[12000:13000])

ax.set(xlabel='time (s)', ylabel='KWH/hh',
       title='Aggregate Household load')
plt.tick_params(rotation=45)
ax.grid()

fig.savefig("Aggregate Household load one month.png")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20,7))
ax.plot(aggLoad.DateTime[12500:12600], aggLoad.AggregateLoad[12500:12600])

ax.set(xlabel='time (s)', ylabel='KWH/hh',
       title='Aggregate Household load ~two days')
plt.tick_params(rotation=45)
ax.grid()

fig.savefig("Aggregate Household load two days.png")
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.plot(aggLoad.DateTime[12500:12550], aggLoad.AggregateLoad[12500:12550])

ax.set(xlabel='time (s)', ylabel='KWH/hh',
       title='Aggregate Household load (one day)')
plt.tick_params(rotation=45)
ax.grid()

fig.savefig("Aggregate Household load (one day).png")
plt.show()

In [ ]:
aggLoad.to_csv('/kaggle/working/aggLoadDataFinal.csv',index=False)

In [ ]:
aggLoad = pd.read_csv('/kaggle/working/aggLoadDataFinal.csv', parse_dates=["DateTime"])
aggLoad

In [ ]:
# Add features useful for time series
!pip install scikit-learn
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing

In [ ]:
# Add features useful for time series
print(aggLoad.info())
# day of week
aggLoad["dayOfWeek"] = aggLoad.DateTime.dt.dayofweek
# day of year
aggLoad["dayOfYear"] = aggLoad.DateTime.dt.dayofyear
# minute of the day
aggLoad["minuteOfDay"] = (aggLoad.DateTime.dt.hour * 60) + aggLoad.DateTime.dt.minute
# number of meters

print(aggLoad.info())

In [ ]:
# Join load data and weather data
mergeData = pd.merge(aggLoad, weatherUpsample, on='DateTime', copy=False)
print(mergeData.info())
mergeData



In [ ]:
# Move first column to the Last
# df = pd.DataFrame(mergeData)
df = mergeData
temp_cols=df.columns.tolist()
new_cols=temp_cols[1:] + temp_cols[0:1]
mergeData=df[new_cols]
print(mergeData)

In [ ]:
from pandas_profiling import ProfileReport

profile = ProfileReport(mergeData, tsmode=True, sortby="DateTime")
profile.to_file('mergeData profile_report.html')

In [ ]:
autoData = mergeData.copy() # keep a copy for the autogluon AutoML

In [ ]:
# Remove the dateTime feature from the dataset

print(mergeData)
mergeData = mergeData.drop(columns=['DateTime'])

In [ ]:
autoData.to_csv('/kaggle/working/autoDataFinal.csv',index=False)

In [ ]:
mergeData.to_csv('/kaggle/working/mergeDataFinal.csv',index=False)

In [ ]:
mergeData = pd.read_csv('/kaggle/working/mergeDataFinal.csv')
mergeData

In [ ]:
mergeData.info()

In [ ]:
# Split the time series data into train, test, and validation datasets
train_size = int(len(mergeData) * 0.7)  # 70% for training
val_size = int(len(mergeData) * 0.2)   # 20% for validation
test_size = len(mergeData) - val_size - train_size  # Remaining for testing

train_data = mergeData[:train_size].copy()
val_data = mergeData[train_size:train_size+val_size].copy()
test_data = mergeData[train_size+val_size:].copy()

print('train_data.head()', train_data.head())
print('val_data.head()', val_data.head())
print('test_data.head()', test_data.head())
print(train_data.info())

num_features = mergeData.shape[1]


In [ ]:
def prediction_plot(testY, test_predict):
      len_prediction=[x for x in range(len(testY))]
      plt.figure(figsize=(20,5))
      plt.plot(len_prediction, testY, marker='.', label="actual")
      plt.plot(len_prediction, test_predict, 'r', label="prediction")
      plt.tight_layout()
      sns.despine(top=True)
      plt.subplots_adjust(left=0.07)
      plt.ylabel('KWH per half hour', size=15)
      plt.xlabel('Time step', size=15)
      plt.legend(fontsize=15)
      plt.show();

In [ ]:
# Normalize the data
train_mean = train_data.mean()
train_std = train_data.std()

train_data = (train_data - train_mean) / train_std
val_data = (val_data - train_mean) / train_std
test_data = (test_data - train_mean) / train_std



In [ ]:
# Visualize distribution of the features
df_std = (mergeData - train_mean) / train_std
df_std = df_std.melt(var_name='Column', value_name='Normalized')
plt.figure(figsize=(12, 6))
ax = sns.violinplot(x='Column', y='Normalized', data=df_std)
_ = ax.set_xticklabels(mergeData.keys(), rotation=90)
plt.savefig('violin_plot.png', format='png')


In [ ]:
class WindowGenerator():
    # https://www.tensorflow.org/tutorials/structured_data/time_series#1_indexes_and_offsets
  def __init__(self, input_width, label_width, shift,
               train_df=train_data, val_df=val_data, test_df=test_data,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

In [ ]:
def split_window(self, features):
    # https://www.tensorflow.org/tutorials/structured_data/time_series#2_split
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window


In [ ]:
def plot(self, model=None, plot_col='AggregateLoad', max_subplots=3):
    # https://www.tensorflow.org/tutorials/structured_data/time_series#3_plot
  inputs, labels = self.example
  plt.figure(figsize=(12, 8))
  plot_col_index = self.column_indices[plot_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(max_n, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]')
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
    else:
      label_col_index = plot_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_col_index],
                edgecolors='k', label='Labels', c='#2ca02c', s=64)
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                  marker='X', edgecolors='k', label='Predictions',
                  c='#ff7f0e', s=64)

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot = plot

In [ ]:
def make_dataset(self, data):
    # https://www.tensorflow.org/tutorials/structured_data/time_series#4_create_tfdatadatasets
  data = np.array(data, dtype=np.float32)
  ds = tf.keras.utils.timeseries_dataset_from_array(
      data=data,
      targets=None,
      sequence_length=self.total_window_size,
      sequence_stride=1,
      shuffle=True,
      seed=randomState,
      batch_size=32,)

  ds = ds.map(self.split_window)

  return ds

WindowGenerator.make_dataset = make_dataset

In [ ]:
# https://www.tensorflow.org/tutorials/structured_data/time_series#4_create_tfdatadatasets
@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result

WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example

In [ ]:
# Prepare data for one-shot multi-step
OUT_STEPS = 48 # 24 hour forecast
IN_STEPS = 336 # look back 1 week
multi_window = WindowGenerator(input_width=IN_STEPS,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS)

multi_window.plot()
multi_window

# Use a naive persistence model as baseline to compare more sophisticated models
Use a 1 week persistence

Georgios Tziolis, Chrysovalantis Spanias, Maria Theodoride, Spyros Theocharides, Javier Lopez-Lorente, Andreas Livera, George Makrides, George E. Georghiou,

Short-term electric net load forecasting for solar-integrated distribution systems based on Bayesian neural networks and statistical post-processing,

Energy,
Volume 271,
2023,
127018,
ISSN 0360-5442,

https://doi.org/10.1016/j.energy.2023.127018.

In [ ]:
# capture performnce of models
multi_val_performance = {}
multi_performance = {}

In [ ]:
# https://www.tensorflow.org/tutorials/structured_data/time_series#linear_model
MAX_EPOCHS = 50

def compile_and_fit(model, window, patience=5):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=[tf.keras.metrics.MeanAbsoluteError()])

  checkpoint_filepath = '/tmp/' + model.name + '/checkpoint'
  checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_filepath, 
                    monitor="val_loss", mode="min", 
                    save_best_only=True, verbose=1)
    
  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping, checkpoint])

  model.load_weights(checkpoint_filepath)
    
  return history


In [ ]:
# Naive 1 week persistence model
NaiveForecast = mergeData.AggregateLoad.copy()

OneWeekNPeriods = 48 * 7

NaiveForecast[:OneWeekNPeriods] = np.nan

for i in range(OneWeekNPeriods, len(mergeData.AggregateLoad)):
    NaiveForecast[i] = mergeData.AggregateLoad[i - OneWeekNPeriods]

print(NaiveForecast.info())
print(NaiveForecast.describe())

In [ ]:
# visualize naive forecast
prediction_plot(mergeData.AggregateLoad, NaiveForecast)

In [ ]:
# visualize naive forecast and actuals for first day of the test dataset
prediction_plot(mergeData.AggregateLoad[train_size+test_size:train_size+test_size+OUT_STEPS], NaiveForecast[train_size+test_size:train_size+test_size+OUT_STEPS])

The average offer price for “balancing actions” – to align supply and demand – between the start of September and early January was £287 a MWh. Data from Elexon, which oversees the market, shows Rye House submitted the 20 highest winter bids – between £5,000 and £6,000 a MWh – for varying volumes of power on 12 December, setting new records
https://www.theguardian.com/business/2023/jan/29/gas-fired-plants-uk-lights-on-cost-profits-energy-crisis

In [ ]:
# Area Under Curve
from scipy.integrate import simpson
from numpy import trapz


# The y values.  A numpy array is used here,
# but a python list could also be used.
# y = np.array([5, 20, 4, 18, 19, 18, 7, 4])
yhat = NaiveForecast[train_size+test_size:train_size+test_size+OUT_STEPS]

# Compute the area using the composite trapezoidal rule.
yhatArea = trapz(yhat, dx=5)
print("yhat area =", yhatArea)

# Compute the area using the composite Simpson's rule.
yhatArea = simpson(yhat, dx=5)
print("yhat area =", yhatArea)

yActual = mergeData.AggregateLoad[train_size+test_size:train_size+test_size+OUT_STEPS]

# Compute the area using the composite trapezoidal rule.
yActualArea = trapz(yActual, dx=5)
print("yActualArea =", yActualArea)

# Compute the area using the composite Simpson's rule.
yActualArea = simpson(yActual, dx=5)
print("yActualArea =", yActualArea)

# Calculate the area difference between the two curves...
AreaDifference = yhatArea - yActualArea
print("AreaDifference =", AreaDifference)

# calculate price at £287 a MWh
cost = (AreaDifference / 1000) * 287
print("cost for 1 day for 5k homes =", cost)


The population of England was 56,489,800 in 2021
24.9 million dwellings in England



In [ ]:
# Scale up costs
cost = cost * (56000000/5000)
print("cost for 1 day for 5k homes =", cost)

In [ ]:

# normalize the naive 
# (NaiveForecast - train_mean) / train_std
NaiveForecastNormed = NaiveForecast.transform(lambda x: (x - train_mean) / train_std)
NaiveForecastNormed.describe()


In [ ]:
# visualize naive forecast and actuals for first day of the test dataset
mergeDataNormed = (mergeData - train_mean) / train_std
prediction_plot(mergeDataNormed.AggregateLoad[train_size+test_size:train_size+test_size+OUT_STEPS], NaiveForecastNormed.AggregateLoad[train_size+test_size:train_size+test_size+OUT_STEPS])

In [ ]:
print(mergeDataNormed.AggregateLoad[train_size:train_size+val_size])
print(NaiveForecastNormed.AggregateLoad[train_size:train_size+val_size])


In [ ]:
# calculate error for naive model (Normed)
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

# calculate error for naive model on validation set
valNaiveMAE = mean_absolute_error(mergeDataNormed.AggregateLoad[train_size:train_size+val_size], NaiveForecastNormed.AggregateLoad[train_size:train_size+val_size])

# calculate error for naive model on test set
testNaiveMAE = mean_absolute_error(mergeDataNormed.AggregateLoad[train_size+val_size:], NaiveForecastNormed.AggregateLoad[train_size+val_size:])

print('valNaiveMAE: ', valNaiveMAE)
print('testNaiveMAE: ', testNaiveMAE)

In [ ]:
# calculate error for naive model (not normed)
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
print('Naive Root Mean Squared Error(RMSE): %.2f; Naive Mean Absolute Error(MAE) : %.2f; Naive Mean Absolute Percantage Error(MAPE) : %.2f '
      % (np.sqrt(mean_squared_error(mergeData.AggregateLoad[OneWeekNPeriods:], NaiveForecast[OneWeekNPeriods:])),
         mean_absolute_error(mergeData.AggregateLoad[OneWeekNPeriods:], NaiveForecast[OneWeekNPeriods:]),
         mean_absolute_percentage_error(mergeData.AggregateLoad[OneWeekNPeriods:], NaiveForecast[OneWeekNPeriods:])))

# calculate error for naive model on validation set
valNaiveMAE = mean_absolute_error(mergeData.AggregateLoad[train_size:train_size+val_size], NaiveForecast[train_size:train_size+val_size])

# calculate error for naive model on test set
testNaiveMAE = mean_absolute_error(mergeData.AggregateLoad[train_size+val_size:], NaiveForecast[train_size+val_size:])

print('valNaiveMAE: ', valNaiveMAE)
print('testNaiveMAE: ', testNaiveMAE)


In [ ]:
# function for plotting the train and test loss curves
def plot_model_loss(history):
    plt.figure(figsize=(8,4))
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epochs')
    plt.legend(loc='upper right')
    plt.show()
    return

In [ ]:
class RepeatBaseline(tf.keras.Model):
  def call(self, inputs):
    return inputs

repeat_baseline = RepeatBaseline()
repeat_baseline.compile(loss=tf.keras.losses.MeanSquaredError(),
                        metrics=[tf.keras.metrics.MeanAbsoluteError()])

multi_val_performance['Naive'] = repeat_baseline.evaluate(mergeDataNormed.AggregateLoad[train_size:train_size+val_size], NaiveForecastNormed.AggregateLoad[train_size:train_size+val_size])
multi_performance['Naive'] = repeat_baseline.evaluate(mergeDataNormed.AggregateLoad[train_size+val_size:], NaiveForecastNormed.AggregateLoad[train_size+val_size:], verbose=0)

# multi_val_performance['Repeat'] = repeat_baseline.evaluate(multi_window.val)
# multi_performance['Repeat'] = repeat_baseline.evaluate(multi_window.test, verbose=0)
# multi_window.plot(repeat_baseline)

In [ ]:
multi_dense_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.Lambda(lambda x: x[:, -1:, :]),
    # Shape => [batch, 1, dense_units]
    tf.keras.layers.Dense(256, activation='relu'),
    # Shape => [batch, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*num_features,
                          kernel_initializer=tf.initializers.zeros()),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, num_features])
])

history = compile_and_fit(multi_dense_model, multi_window)
multi_dense_model.save('multi_dense_model.keras')
plot_model_loss(history)

# IPython.display.clear_output()
multi_val_performance['Dense'] = multi_dense_model.evaluate(multi_window.val)
multi_performance['Dense'] = multi_dense_model.evaluate(multi_window.test, verbose=0)
multi_window.plot(multi_dense_model)

# CNN

In [ ]:
CONV_WIDTH = 10
multi_conv_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, CONV_WIDTH, features]
    tf.keras.layers.Lambda(lambda x: x[:, -CONV_WIDTH:, :]),
    # Shape => [batch, 1, conv_units]
    tf.keras.layers.Conv1D(256, activation='relu', kernel_size=(CONV_WIDTH)),
    # Shape => [batch, 1,  out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*num_features,
                          kernel_initializer=tf.initializers.zeros()),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, num_features])
])

history = compile_and_fit(multi_conv_model, multi_window)
multi_conv_model.save('multi_conv_model.keras')
plot_model_loss(history)
# IPython.display.clear_output()

multi_val_performance['Conv'] = multi_conv_model.evaluate(multi_window.val)
multi_performance['Conv'] = multi_conv_model.evaluate(multi_window.test, verbose=0)
multi_window.plot(multi_conv_model)


In [ ]:
multi_rnn_model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(256, return_sequences=False),
    # Shape => [batch, out_steps*features].
    tf.keras.layers.Dense(OUT_STEPS*num_features,
                          kernel_initializer=tf.initializers.zeros()),
    # Shape => [batch, out_steps, features].
    tf.keras.layers.Reshape([OUT_STEPS, num_features])
])

history = compile_and_fit(multi_rnn_model, multi_window)
plot_model_loss(history)
multi_rnn_model.save('multi_rnn_model.keras')
# IPython.display.clear_output()

multi_val_performance['RNN'] = multi_rnn_model.evaluate(multi_window.val)
multi_performance['RNN'] = multi_rnn_model.evaluate(multi_window.test, verbose=0)
multi_window.plot(multi_rnn_model)

# LSTM Model

In [ ]:
multi_lstm_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, lstm_units].
    # Adding more `lstm_units` just overfits more quickly.
    tf.keras.layers.LSTM(256, return_sequences=False),
    # Shape => [batch, out_steps*features].
    tf.keras.layers.Dense(OUT_STEPS*num_features,
                          kernel_initializer=tf.initializers.zeros()),
    # Shape => [batch, out_steps, features].
    tf.keras.layers.Reshape([OUT_STEPS, num_features])
])

start_time = time.time()
history = compile_and_fit(multi_lstm_model, multi_window)
print('%s seconds' % (time.time() - start_time))
multi_lstm_model.save('multi_lstm_model.keras')
plot_model_loss(history)
# IPython.display.clear_output()

multi_val_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.val)
multi_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.test, verbose=0)
multi_window.plot(multi_lstm_model)


# CNN-LSTM

In [ ]:
x = np.arange(len(multi_performance))
width = 0.3

metric_name = 'mean_absolute_error'
metric_index = multi_lstm_model.metrics_names.index('mean_absolute_error')
val_mae = [v[metric_index] for v in multi_val_performance.values()]
test_mae = [v[metric_index] for v in multi_performance.values()]

plt.bar(x - 0.17, val_mae, width, label='Validation')
plt.bar(x + 0.17, test_mae, width, label='Test')
plt.xticks(ticks=x, labels=multi_performance.keys(),
           rotation=45)
plt.ylabel(f'MAE (average over all times and outputs)')
_ = plt.legend()
plt.savefig('model performance.png', format='png')


In [ ]:
# build a single record dataset from start of the test dataset
print(mergeDataNormed)
print(train_size+test_size-IN_STEPS)
print(mergeDataNormed.iloc[train_size+test_size-IN_STEPS:train_size+test_size])

ds = tf.keras.utils.timeseries_dataset_from_array(
      data=test_data[:IN_STEPS],
      targets=None,
      sequence_length=IN_STEPS,
      sequence_stride=1,
      shuffle=False,
      batch_size=32,)

testYhatNormed = multi_lstm_model.predict(ds)

#     data=test_data[:IN_STEPS],
# data=mergeDataNormed[train_size+test_size-IN_STEPS:train_size+test_size],

In [ ]:
print(testYhatNormed)
print(testYhatNormed[0,:,-1])

In [ ]:
# Invert stabdardization
testYhat = (np.array(testYhatNormed) * np.array(train_std)) + np.array(train_mean)

print(testYhat)
print(testYhat[0,:,-1])

In [ ]:
# CNN-LSTM

# Plot of naive, LSTM and actuals

In [ ]:
      len_prediction=[x for x in range(len(testYhat[0,:,-1]))]
      plt.figure(figsize=(10,5))
      # plt.plot(len_prediction, test_data[:OUT_STEPS].AggregateLoad, marker='.', label="actual")
      plt.plot(len_prediction, mergeDataNormed.AggregateLoad[train_size+val_size:train_size+val_size+OUT_STEPS], marker='.', label="actual")
      plt.plot(len_prediction, testYhatNormed[0,:,-1], 'r', label="LSTM prediction")
      plt.plot(len_prediction, NaiveForecastNormed[train_size+val_size:train_size+val_size+OUT_STEPS].AggregateLoad, 'g', label="Naive prediction")
        
      plt.tight_layout()
      sns.despine(top=True)
      plt.subplots_adjust(left=0.07)
      plt.ylabel('KWH per half hour (Normed)', size=15)
      plt.xlabel('Time step', size=15)
      plt.legend(fontsize=15)
      plt.show();
      plt.savefig('Test First Day Normed.png', format='png')

    
    

In [ ]:
      len_prediction=[x for x in range(len(testYhat[0,:,-1]))]
      plt.figure(figsize=(10,5))
      # plt.plot(len_prediction, test_data[:OUT_STEPS].AggregateLoad, marker='.', label="actual")
      plt.plot(len_prediction, mergeData.AggregateLoad[train_size+val_size:train_size+val_size+OUT_STEPS], marker='.', label="actual")
      plt.plot(len_prediction, testYhat[0,:,-1], 'r', label="LSTM prediction")
      plt.plot(len_prediction, NaiveForecast[train_size+val_size:train_size+val_size+OUT_STEPS], 'g', label="Naive prediction")
        
      plt.tight_layout()
      sns.despine(top=True)
      plt.subplots_adjust(left=0.07)
      plt.ylabel('KWH per half hour', size=15)
      plt.xlabel('Time step', size=15)
      plt.legend(fontsize=15)
      plt.show()
      plt.savefig('Test First Day.png', format='png')



# Estimate financial impact of improved forecast

In [ ]:
# Area Under Curve
from scipy.integrate import simpson
from numpy import trapz



In [ ]:
# install AutoGluon AutoML
!pip install autogluon
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

In [ ]:
autoData = pd.read_csv('/kaggle/working/autoDataFinal.csv')
autoData

In [ ]:
# AutoGluon specific data preparation
# Split the time series data into train, test, and validation datasets
train_size = int(len(autoData) * 0.7)  # 70% for training
val_size = int(len(autoData) * 0.2)   # 20% for validation
test_size = len(autoData) - val_size - test_size  # Remaining for testing

ag_train_data = autoData[:train_size]
ag_val_data = autoData[train_size:train_size+val_size]
ag_test_data = autoData[train_size+val_size:]

print('ag_train_data:\n', ag_train_data)
print('ag_val_data:\n', ag_val_data)
print('ag_test_data\n', ag_test_data)
# print(ag_train_data.info())

# AutoGluon requires an ItemID Column, so adding one...
ag_train_data['item_id'] = 'LoadSum'
ag_train_data = ag_train_data.astype({"item_id": str})
ag_val_data['item_id'] = 'LoadSum'
ag_val_data = ag_val_data.astype({"item_id": str})
ag_test_data['item_id'] = 'LoadSum'
ag_test_data = ag_test_data.astype({"item_id": str})


In [ ]:
# take a quick look at the split datasets
print('ag_train_data\n', ag_train_data)
print('ag_val_data\n', ag_val_data)
print('ag_test_data\n', ag_test_data)


In [ ]:
# load training data in to required AutoGluon proprietary data frame
# print(ag_train_data.info())
ag_train_data_tsdf = TimeSeriesDataFrame.from_data_frame(
    ag_train_data,
    id_column="item_id",
    timestamp_column="DateTime"
)
ag_train_data_tsdf

In [ ]:
# load validation data in to required AutoGluon proprietary data frame, "_tsdf" suffix = time series data frame
ag_val_data_tsdf = TimeSeriesDataFrame.from_data_frame(
    ag_val_data,
    id_column="item_id",
    timestamp_column="DateTime"
)
ag_val_data_tsdf

In [ ]:
# load test data in to required AutoGluon proprietary data frame
# print(ag_test_data.info())
ag_test_data_tsdf = TimeSeriesDataFrame.from_data_frame(
    ag_test_data,
    id_column="item_id",
    timestamp_column="DateTime"
)
ag_test_data_tsdf

In [ ]:
# at "high_quality" level, training takes about 45 minutes...
# training takes about 15 minutes for DeepAR
# training takes about 21 minutes for TemporalFusionTransformer
# training takes about 4 minutes for PatchTST
# training takes about 4 minutes for PatchTST

In [ ]:
ag_predictor = TimeSeriesPredictor(
    prediction_length=48,
    path="autogluon-london-half-hourly",
    target="AggregateLoad",
    eval_metric="MASE",
)

ag_predictor.fit(
    ag_train_data_tsdf,
    presets="medium_quality",
    time_limit=6000,
)

In [ ]:
# The test score is computed using the last
# prediction_length=48 timesteps of each time series in test_data
ag_predictor.leaderboard(ag_val_data_tsdf, silent=True)

In [ ]:
# generate predictions
ag_predictions = ag_predictor.predict(ag_val_data_tsdf)
ag_predictions.head()

In [ ]:
print(train_mean)

In [ ]:
# standardize the predictions so we can compare prediction errors with other models
ag_predictionsNormed = (ag_predictions["mean"] - train_mean["AggregateLoad"]) / train_std["AggregateLoad"]
ag_predictionsNormed

In [ ]:
# plot prediction results, history and actual test data values
plt.figure(figsize=(20, 3))

item_id = "LoadSum"
y_past = ag_val_data_tsdf.loc[item_id]["AggregateLoad"]
y_pred = ag_predictions.loc[item_id]
y_val = ag_test_data_tsdf.loc[item_id]["AggregateLoad"]
# y_val = ag_val_data_tsdf.loc[item_id]["AggregateLoad"]

plt.plot(y_past[-100:], label="Past time series values")
plt.plot(y_pred["mean"], label="Mean forecast")
plt.plot(y_val[:48], label="Future time series values")

plt.fill_between(
    y_pred.index, y_pred["0.1"], y_pred["0.9"], color="red", alpha=0.1, label=f"10%-90% confidence interval"
)
plt.legend();
plt.savefig('AutoML forecast.png', format='png')


In [ ]:
# load test data in to required AutoGluon proprietary data frame
# print(ag_test_data.info())
ag_testday1_data_tsdf = TimeSeriesDataFrame.from_data_frame(
    ag_val_data[-IN_STEPS:],
    id_column="item_id",
    timestamp_column="DateTime"
)
ag_testday1_data_tsdf

In [ ]:
# generate predictions
agTestDay1_predictions = ag_predictor.predict(ag_testday1_data_tsdf)
agTestDay1_predictions.head()

In [ ]:
agTestDay1_predictions

In [ ]:
      len_prediction=[x for x in range(len(testYhat[0,:,-1]))]
      plt.figure(figsize=(10,5))
      # plt.plot(len_prediction, test_data[:OUT_STEPS].AggregateLoad, marker='.', label="actual")
      plt.plot(len_prediction, mergeData.AggregateLoad[train_size+val_size:train_size+val_size+OUT_STEPS], marker='.', label="actual")
      plt.plot(len_prediction, testYhat[0,:,-1], 'r', label="LSTM prediction")
      plt.plot(len_prediction, NaiveForecast[train_size+val_size:train_size+val_size+OUT_STEPS], 'g', label="Naive prediction")
      plt.plot(len_prediction, agTestDay1_predictions["mean"], 'y', label="AutoML prediction")
        
      plt.tight_layout()
      sns.despine(top=True)
      plt.subplots_adjust(left=0.07)
      plt.ylabel('KWH per half hour', size=15)
      plt.xlabel('Time step', size=15)
      plt.legend(fontsize=15)
      plt.show()
      plt.savefig('Test First Day.png', format='png')


In [ ]:
# calculate error for naive model on validation set
# valAutoMLMAE = mean_absolute_error(mergeDataNormed.AggregateLoad[train_size:train_size+val_size], ag_predictionsNormed[train_size:train_size+val_size])

# calculate error for naive model on test set
testAutoMLMAE = mean_absolute_error(mergeDataNormed.AggregateLoad[train_size+val_size:], ag_predictionsNormed)

# print('valAutoMLMAE: ', valAutoMLMAE)
print('testAutoMLMAE: ', testAutoMLMAE)

In [ ]:
import dill
# save
dill.dump_session('notebook_env.db')
